# TSTR GM Dataset E

In [2]:
#import libraries
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import os
print('Libraries imported!!')

Libraries imported!!


In [3]:
#define directory of functions and actual directory
HOME_PATH = '' #home path of the project
FUNCTIONS_DIR = 'EVALUATION FUNCTIONS/UTILITY'
ACTUAL_DIR = os.getcwd()

#change directory to functions directory
os.chdir(HOME_PATH + FUNCTIONS_DIR)

#import functions for data labelling analisys
from utility_evaluation import DataPreProcessor
from utility_evaluation import train_evaluate_model

#change directory to actual directory
os.chdir(ACTUAL_DIR)
print('Functions imported!!')

Functions imported!!


## 1. Read data

In [4]:
#read real dataset
train_data = pd.read_csv(HOME_PATH + 'SYNTHETIC DATASETS/GM/E_PimaIndiansDiabetes_Synthetic_GM.csv')
categorical_columns = ['Outcome']
for col in categorical_columns :
    train_data[col] = train_data[col].astype('category')
train_data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,11,140,67,-6,139,21.454652,0.138648,45,0
1,0,111,63,0,108,23.818868,0.347041,25,0
2,5,88,77,14,-64,27.547578,0.331241,63,1
3,3,94,79,25,120,32.149746,0.164218,38,1
4,9,131,0,29,244,30.608647,0.296024,25,1
...,...,...,...,...,...,...,...,...,...
609,4,132,64,37,-43,40.607103,0.863559,42,1
610,7,114,79,20,-7,29.984722,0.427422,21,0
611,1,68,71,1,-89,23.307341,0.261145,22,0
612,6,97,73,32,257,27.208406,0.176116,29,0


In [6]:
#read test data
test_data = pd.read_csv(HOME_PATH + 'REAL DATASETS/TEST DATASETS/E_PimaIndiansDiabetes_Real_Test.csv')
for col in categorical_columns :
    test_data[col] = test_data[col].astype('category')
test_data

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,98,58,33,190,34.0,0.430,43,0
1,2,112,75,32,0,35.7,0.148,21,0
2,2,108,64,0,0,30.8,0.158,21,0
3,8,107,80,0,0,24.6,0.856,34,0
4,7,136,90,0,0,29.9,0.210,50,0
...,...,...,...,...,...,...,...,...,...
149,9,165,88,0,0,30.4,0.302,49,1
150,1,77,56,30,56,33.3,1.251,24,0
151,8,95,72,0,0,36.8,0.485,57,0
152,2,146,70,38,360,28.0,0.337,29,1


In [7]:
target = 'Outcome'
#quick look at the breakdown of class values
print('Train data')
print(train_data.shape)
print(train_data.groupby(target).size())
print('#####################################')
print('Test data')
print(test_data.shape)
print(test_data.groupby(target).size())

Train data
(614, 9)
Outcome
0    378
1    236
dtype: int64
#####################################
Test data
(154, 9)
Outcome
0    99
1    55
dtype: int64


## 2. Pre-process training data

In [8]:
target = 'Outcome'
categorical_columns = None
numerical_columns = train_data.select_dtypes(include=['int64','float64']).columns.tolist()
categories = None

data_preprocessor = DataPreProcessor(categorical_columns, numerical_columns, categories)
x_train = data_preprocessor.preprocess_train_data(train_data.loc[:, train_data.columns != target])
y_train = train_data.loc[:, target]

x_train.shape, y_train.shape

((614, 8), (614,))

## 3. Preprocess test data

In [9]:
x_test = data_preprocessor.preprocess_test_data(test_data.loc[:, test_data.columns != target])
y_test = test_data.loc[:, target]
x_test.shape, y_test.shape

((154, 8), (154,))

## 4. Create a dataset to save the results

In [10]:
results = pd.DataFrame(columns = ['model','accuracy','precision','recall','f1'])
results

,model,accuracy,precision,recall,f1


## 4. Train and evaluate Random Forest Classifier

In [11]:
rf_results = train_evaluate_model('RF', x_train, y_train, x_test, y_test)
results = results.append(rf_results, ignore_index=True)
rf_results

[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.1s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=3)]: Using backend ThreadingBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    0.0s
[Parallel(n_jobs=3)]: Done 100 out of 100 | elapsed:    0.0s finished


,model,accuracy,precision,recall,f1
0,RF,0.7013,0.692,0.7013,0.6935


## 5. Train and Evaluate KNeighbors Classifier

In [12]:
knn_results = train_evaluate_model('KNN', x_train, y_train, x_test, y_test)
results = results.append(knn_results, ignore_index=True)
knn_results

,model,accuracy,precision,recall,f1
0,KNN,0.6688,0.6496,0.6688,0.6456


## 6. Train and evaluate Decision Tree Classifier

In [13]:
dt_results = train_evaluate_model('DT', x_train, y_train, x_test, y_test)
results = results.append(dt_results, ignore_index=True)
dt_results

,model,accuracy,precision,recall,f1
0,DT,0.5649,0.5844,0.5649,0.5719


## 7. Train and evaluate Support Vector Machines Classifier

In [14]:
svm_results = train_evaluate_model('SVM', x_train, y_train, x_test, y_test)
results = results.append(svm_results, ignore_index=True)
svm_results

[LibSVM]

,model,accuracy,precision,recall,f1
0,SVM,0.4286,0.5013,0.4286,0.4306


## 8. Train and evaluate Multilayer Perceptron Classifier

In [15]:
mlp_results = train_evaluate_model('MLP', x_train, y_train, x_test, y_test)
results = results.append(mlp_results, ignore_index=True)
mlp_results

Iteration 1, loss = 0.68484064
Iteration 2, loss = 0.65541569
Iteration 3, loss = 0.63949780
Iteration 4, loss = 0.62799095
Iteration 5, loss = 0.61885416
Iteration 6, loss = 0.60772454
Iteration 7, loss = 0.59656989
Iteration 8, loss = 0.58602534
Iteration 9, loss = 0.57833457
Iteration 10, loss = 0.57322182
Iteration 11, loss = 0.56910037
Iteration 12, loss = 0.56552303
Iteration 13, loss = 0.56296187
Iteration 14, loss = 0.55927641
Iteration 15, loss = 0.55721027
Iteration 16, loss = 0.55435083
Iteration 17, loss = 0.55176047
Iteration 18, loss = 0.55064236
Iteration 19, loss = 0.55039598
Iteration 20, loss = 0.54813922
Iteration 21, loss = 0.54510657
Iteration 22, loss = 0.54297334
Iteration 23, loss = 0.53980042
Iteration 24, loss = 0.53779050
Iteration 25, loss = 0.53561530
Iteration 26, loss = 0.53511512
Iteration 27, loss = 0.53031604
Iteration 28, loss = 0.52822206
Iteration 29, loss = 0.52702815
Iteration 30, loss = 0.52463749
Iteration 31, loss = 0.52716848
Iteration 32, los

,model,accuracy,precision,recall,f1
0,MLP,0.6299,0.6314,0.6299,0.6306


## 9. Save results file

In [16]:
results.to_csv('RESULTS/models_results_gm.csv', index=False)
results

,model,accuracy,precision,recall,f1
0,RF,0.7013,0.6920,0.7013,0.6935
1,KNN,0.6688,0.6496,0.6688,0.6456
2,DT,0.5649,0.5844,0.5649,0.5719
3,SVM,0.4286,0.5013,0.4286,0.4306
4,MLP,0.6299,0.6314,0.6299,0.6306
